### Homework 8 for Computational Economics, ECON-GA 3002

**{Name, NYUID, Email}**: Arnav Sood, N11193569, asood@nyu.edu

#### Exercise 1:

> If $S$ is a discrete set, show that if the set of distributions on $S$, $\mathbb{P}(S)$, is compact in $S$, then $S$ is finite.

**Proof**: Argue contrapositively: that is, show $|S| = |\mathbb{N}| = \aleph_{0} \implies \mathbb{P}(S)$ not compact. (Sequential) compactness requires that every sequence in a subset $K$ contains a subsequence which converges to a point in $S$. Let $K$ be the set of delta distributions $(1, 0, 0, ...), (0, 1, 0, ...)$, and so on. These form a subset of $S$, but clearly no subsequence converges, as the $l_1$  norm of their difference of any two of them is always 2. Therefore, $\mathbb{P}(S)$ compact shows $S$ finite for discrete $S$.


#### Exercise 2:

> Show that the deterministic Markov chain given by $X_{t+1} = X_t + 1$ has no stationary distribution.

**Proof:** We can uniquely identify each state $\gamma \in \mathbb{Z}$ with a probability distribution $p_{\gamma} \in \mathbb{P}(S)$. This is because $p_{\gamma}$ is the dirac distibrution which assigns mass 1 to $\gamma + 1$ and weight 0 everywhere else, and $\gamma + 1$ is uniquely identified with $\gamma$. So, a stationary distribution which maps to itself would correspond to a state $\gamma$ which maps to itself, which would require $\gamma + 1 = \gamma$, which is a contradiction. Therefore, no stationary distribution exists.

#### Exercise 3:

> Consider a Markov process given by the following dynamics. Select an integer $Q$, and an integer $0 \leq q \leq Q$. At time $t$, a store's inventory $X_t$ is some integer. The store then orders inventory $Q - X_t$ subject to a threshold rule: if $X_t \leq q$ the store orders $Q - X_t$, and if not the store orders nothing. The store is then subject to a stochastic demand, IID with $\text{Pr}[D = d]$ given by $\frac{1}{2}^{d+1}$ for $d \in \{0, 1, ...\}$. The store meets the demand as best it can without exceeding its current stock. The remainder, zero or nonzero, forms the next period's inventory $X_{t+1}$. 

> Show that the above is globally stable $\forall Q, 0 \leq q \leq Q$.

**Proof**: Take any finite $Q$. Setting $0 \leq q \leq Q$ yields a finite state space of cardinality $Q+1$. The associated Markov chain is irreducible, as given any initial state $\gamma$, there is positive probability on the event of selling all one's inventory, then selling some $Q-\epsilon$ for arbitrary $\epsilon \in \{0, 1, ..., Q\}$.

It is also aperiodic, as any state is accessible from itself in any number of steps greater than 3. The associated probability is, at least, the probability associated with selling everything, then not selling anything until time $k-1$, until you sell $Q-\gamma$ for initial state $\gamma$, giving you inventory $\gamma$ at time $k$.

Since the Markov chain associated with $Q, q$ is aperiodic and irreducible on a finite state space, we know it is globally stable.


#### Exercise 4

> Let $\psi$ be the stationary distribution corresponding to $(Q, q) = (5, 2)$. Using QuantEcon's $\texttt{MarkovChain}$ class, compute $\psi$.

> Import dependencies and define a `primitives` type for the model. The idea for doing so came from Alberto Polo's homework 7.

In [1]:
using QuantEcon
type primitives
    q::Int64
    Q::Int64
    P::Array{Float64,2}
    primitives(q,Q) = q > Q ? error("q out of range") : new(q,Q,zeros(Q+1,Q+1))
end

> Write a function to compute the stochastic matrix for a given set of primitives. 

> The `transitionProbability` function computes $p(x,y)$ for all $x$, and nonzero $y$. The reason that we restrict to nonzero $y$ is that the 0 inventory case corresponds to an infinite number of states (i.e., all $d > Q$). We could use the geometric series formula, but since all the corrections come from one column, an ex post correction is cleaner.

In [2]:
function getStochasticKernel(primitives)
    
    P = primitives.P
    Q = primitives.Q
    q = primitives.q
    
    # Define transitionProbability
    function transitionProbability(x,y)
        if x <= q
            return (0.5)^(Q-y+1)
        elseif (x > q) & (y <= x)
            return (0.5)^(x-y+1)
        else
            return 0
        end
    end
    
    # Fill for nonzero targets
    for row in 1:Q+1
        for column in 2:Q+1
            P[row,column] = transitionProbability(row-1,column-1)
        end
    end
    
    # Fill in 0 targets
    for row in 1:Q+1
        P[row,1] = 1 - sum(P[row,:])
    end
    
end

getStochasticKernel (generic function with 1 method)

> Write a wrapper function to take a set of primitives and return the stationary distribution and stochastic kernel.

In [3]:
function computeStationary(primitives)
    getStochasticKernel(primitives)
    inventoryChain = MarkovChain(primitives.P)
    return mc_compute_stationary(inventoryChain)
end

computeStationary (generic function with 1 method)

> Try it for the primitives in this exercise.

In [4]:
exercise4 = primitives(2,5)
println("The stationary distribution is: \n", computeStationary(exercise4))

The stationary distribution is: 
[0.0625,0.0625,0.125,0.25,0.25,0.25]


#### Exercise 5

> Compute the stationary distribution from above, but use an explicit iteration on the $l_1$ norm instead of a package from QuantEcon.

> We do this by rewriting our `computeStationary` function to account for some new parameters (tolerance bound (default $10^{-4}$, maximum iterations (default 500)). Some other notes:

> We use the rational type `1//Q` in Julia to allow us to get the uniform distribution over arbitrary `Q`, and the `'` operator gives us a row vector.

> The idea came from, in part, Timothy Petliar's homework 7.

In [5]:
function computeStationary(primitives,bound=0.0001,maxIter=500)
    getStochasticKernel(primitives)
    Q = primitives.Q
    P = primitives.P
    q = primitives.q
    currentGuess=(zeros(Q+1)+1//(Q+1))'
    
    function computeNorm(dist1,dist2)
        return sum(abs(dist1-dist2))
    end
    
    currDistance = computeNorm(currentGuess*P,currentGuess)
    iterCount = 0
    
    while iterCount <= maxIter
        if currDistance < bound
            println("The stationary distribution is approximately: \n")
            println(currentGuess)
            return
        else
            currentGuess = currentGuess*P
            currDistance = computeNorm(currentGuess*P,currentGuess)
            iterCount+=1
        end
    end
    
    println("The stationary distribution is approximately: \n", currentGuess)
    error("Error: Maximum Iteration Count Exceeded \n")

end

computeStationary (generic function with 3 methods)

> Try it.

In [6]:
exercise5 = primitives(2,5)
computeStationary(exercise5,0.0001,500)

The stationary distribution is approximately: 

[0.0625076542297999 0.0625076542297999 0.1250153084595998 0.2500306169191996 0.2499885956446329 0.24995017051696772]


#### Exercise 6

> Run for $q \in \{2, 5, 10, 15\}, Q = 20$. Plot the stationary distributions, and comment.

> First, reset computeStationary to QuantEcon's implementation, and suppress printing.

In [7]:
function computeStationary(primitives)
    getStochasticKernel(primitives)
    inventoryChain = MarkovChain(primitives.P)
    return mc_compute_stationary(inventoryChain)
end

computeStationary (generic function with 3 methods)

> Then, write a function to generate the results.

In [8]:
function runExperiments(qVector=[2,5,10,15],Q=20)
        results = Array{Float64}(length(qVector),Q+1)
        for i in 1:size(results)[1]
                results[i,:] = computeStationary(primitives(qVector[i],Q))
        end
        return results
end

runExperiments (generic function with 3 methods)

> Run and plot. The abstruse-looking type in our `data` array is to enable it to hold Plotly traces, which are of that type.

In [ ]:
# Not ready to run yet.

using PlotlyJS
using Blink

const Q = 20
const qVec = [2,5,10,15]

results = runExperiments(qVec,Q)

data = Array{PlotlyJS.GenericTrace{Dict{Symbol,Any}}}(length(qVec))
xVec = collect(1:Q+1)
for i in 1:length(qVec)
    data[i] = scatter(;x=xVec,y=results[i])
end

plot(data)